In [8]:
import pandas as pd
import pyarrow.parquet as pq
import random

Create a toy dataset which has only half of the intensities to test changed output layer

In [14]:
inpath = '/cmnfs/proj/prosit_astral/bmpc_dlomix_group/datasets/parquet/noptm_baseline_small'
outpath = '/cmnfs/proj/prosit_astral/bmpc_dlomix_group/datasets/parquet/transfer_learning_toy_dataset/half_intesities'
dataset_types = ['train', 'val', 'test'] 

for data_type in dataset_types:
    dataset = pd.read_parquet(f'{inpath}_{data_type}.parquet')
    # cut number of intensities in half
    dataset.loc[:, 'intensities_raw'] = dataset.loc[:, 'intensities_raw'].apply(lambda intensities: intensities[:87])
    dataset.reset_index(inplace=True)
    dataset.drop(columns=['index'], inplace=True)
    dataset.to_parquet(f'{outpath}_{data_type}.parquet', index=False)

Create a toy dataset which has a the arbitrary modification M[UNIMOD:999] in some sequences

In [1]:
inpath = '/cmnfs/proj/prosit_astral/bmpc_dlomix_group/datasets/parquet/noptm_baseline_small'
outpath = '/cmnfs/proj/prosit_astral/bmpc_dlomix_group/datasets/parquet/transfer_learning_toy_dataset/new_modification'
dataset_types = ['train', 'val', 'test'] 

In [49]:
def insert_modification(seq, prob: float = 1):
    if 'UNIMOD' in seq:
        return seq
    if random.random() > prob:
        return seq
    
    idx = random.randint(3, len(seq) - 3)
    return seq[:idx] + 'M[UNIMOD:999]' + seq[idx:]

In [56]:
for data_type in dataset_types:
    dataset = pd.read_parquet(f'{inpath}_{data_type}.parquet')
    # insert the arbitrary modification M[UNIMOD:999] in 10% of unmodified sequences
    dataset.loc[:, 'modified_sequence'] = dataset.loc[:, 'modified_sequence'].apply(insert_modification, prob=0.1)
    dataset.reset_index(inplace=True)
    dataset.drop(columns=['index'], inplace=True)
    dataset.to_parquet(f'{outpath}_{data_type}.parquet', index=False)